In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,4,7'

In [2]:
from src.ScoreBasedInferenceModel import ScoreBasedInferenceModel as SBIm

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import torch

from scipy.stats import norm, gaussian_kde
from scipy import optimize
import scipy

import os

%matplotlib inline

# Load data

In [3]:
data_model = "842"

In [4]:
# --- Clean the data ---
# Chempy sometimes returns zeros or infinite values, which need to removed
def clean_data(x, y):
    # Remove all zeros from the training data
    index = np.where((y == 0).all(axis=1))[0]
    x = np.delete(x, index, axis=0)
    y = np.delete(y, index, axis=0)

    # Remove all infinite values from the training data
    index = np.where(np.isfinite(y).all(axis=1))[0]
    x = x[index]
    y = y[index]

    # Remove H from Elements
    y = np.delete(y, 2, 1)

    return x, y

In [5]:
# ---  Load in the validation data ---
path_test = os.getcwd() + f'/data/Chempy_model_comp_data/chempy_{data_model}_val.npz'
val_data = np.load(path_test, mmap_mode='r')

val_x = val_data['params']
val_y = val_data['abundances']

val_x, val_y = clean_data(val_x, val_y)

# convert to torch tensors
val_x = torch.tensor(val_x, dtype=torch.float32)
val_y = torch.tensor(val_y, dtype=torch.float32)

# --- add noise ---
pc_ab = 5 # percentage error in abundance

val_y_err = torch.ones_like(val_y)*float(pc_ab)/100.
val_y = norm.rvs(loc=val_y,scale=val_y_err)
val_y = torch.tensor(val_y).float()

val_data = torch.cat((val_x, val_y), 1)

## Define ModelTransfuser

In [6]:
# Define the ModelTransfuser
model_442 = SBIm.load('data/models/chempy_442/Model_checkpoint.pt')
model_742 = SBIm.load('data/models/chempy_742/Model_checkpoint.pt')
model_842 = SBIm.load('data/models/chempy_842/Model_checkpoint.pt')

## Sampling

In [7]:
posterior_mask = torch.zeros_like(val_data[0])
posterior_mask[6:] = 1

data = val_data[:1000, posterior_mask.type(torch.bool)]

In [8]:
test_index=10

In [9]:
null_hypothesis_442 = model_442.sample(torch.zeros(14), condition_mask=torch.zeros(14), device="cuda:0", timesteps=100, cfg_alpha=None, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
null_hypothesis_742 = model_742.sample(torch.zeros(14), condition_mask=torch.zeros(14), device="cuda:0", timesteps=100, cfg_alpha=None, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
null_hypothesis_842 = model_842.sample(torch.zeros(14), condition_mask=torch.zeros(14), device="cuda:0", timesteps=100, cfg_alpha=None, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")

100%|██████████| 1/1 [00:01<00:00,  1.91s/it]


In [10]:
posterior_442 = model_442.sample(data[test_index], condition_mask=posterior_mask, device="cuda:0", timesteps=100, cfg_alpha=None, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
posterior_742 = model_742.sample(data[test_index], condition_mask=posterior_mask, device="cuda:0", timesteps=100, cfg_alpha=None, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
posterior_842 = model_842.sample(data[test_index], condition_mask=posterior_mask, device="cuda:0", timesteps=100, cfg_alpha=None, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.85s/it]


In [25]:
posterior_mask.bool()

tensor([False, False, False, False, False, False,  True,  True,  True,  True,
         True,  True,  True,  True])

In [28]:
posterior_442[:,:,(1-posterior_mask).bool()]

tensor([[[-2.3169, -2.9535,  0.1727,  0.5660,  0.4562,  6.7417],
         [-2.3815, -3.0777, -0.4272,  0.4362,  0.4904, 13.1684],
         [-2.1150, -2.5096, -0.3913,  0.5760,  0.4802,  9.3550],
         ...,
         [-2.3705, -2.6471,  0.1634,  0.5051,  0.7053,  8.1304],
         [-2.1569, -2.6800,  0.0134,  0.6350,  0.5855,  6.9629],
         [-2.2496, -2.6100,  0.1382,  0.7796,  0.5011,  6.1941]]])

In [11]:
theta_hat_442 = posterior_442.mean(dim=1)[:,:6]
theta_hat_742 = posterior_742.mean(dim=1)[:,:6]
theta_hat_842 = posterior_842.mean(dim=1)[:,:6]

In [12]:
print(f"Ground Truth: {val_data[test_index,:6]}")
print(f"Model 442: {theta_hat_442}")
print(f"Model 742: {theta_hat_742}")
print(f"Model 842: {theta_hat_842}")

Ground Truth: tensor([-2.2482, -2.7791, -0.0193,  0.6013,  0.4290, 10.8796])
Model 442: tensor([[-2.2340, -2.7413, -0.1437,  0.5273,  0.4616,  8.3242]])
Model 742: tensor([[-1.6914, -3.0892, -0.0119,  0.6491,  0.5516,  8.4723]])
Model 842: tensor([[-2.1733, -2.6000, -0.1935,  0.5262,  0.4630,  8.1301]])


In [13]:
likelihood_mask = torch.zeros_like(val_data[0])
likelihood_mask[:6] = 1

In [14]:
likelihood_442 = model_442.sample(theta_hat_442, condition_mask=likelihood_mask, device="cuda:0", timesteps=100, cfg_alpha=None, save_trajectory=True, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
likelihood_742 = model_742.sample(theta_hat_742, condition_mask=likelihood_mask, device="cuda:0", timesteps=100, cfg_alpha=None, save_trajectory=True, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
likelihood_842 = model_842.sample(theta_hat_842, condition_mask=likelihood_mask, device="cuda:0", timesteps=100, cfg_alpha=None, save_trajectory=True, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")

100%|██████████| 1/1 [00:01<00:00,  1.86s/it]


In [15]:
x_hat_442 = likelihood_442.mean(dim=1)[:,6:]
x_hat_742 = likelihood_742.mean(dim=1)[:,6:]
x_hat_842 = likelihood_842.mean(dim=1)[:,6:]

In [16]:
print(f"Ground Truth: {val_data[test_index,6:]}")
print(f"Model 442: {x_hat_442}")
print(f"Model 742: {x_hat_742}")
print(f"Model 842: {x_hat_842}")

Ground Truth: tensor([-0.0728,  0.2306, -0.3019,  0.1014,  0.0569,  0.4863,  0.2233,  0.1436])
Model 442: tensor([[ 0.0066,  0.3272, -0.2645,  0.0880,  0.0804,  0.4275,  0.3083,  0.2780]])
Model 742: tensor([[-0.1518,  0.2781, -0.0285, -0.0759,  0.2716,  0.1481,  0.2128,  0.2581]])
Model 842: tensor([[-0.0335,  0.3066, -0.2292,  0.1254,  0.0293,  0.4364,  0.2202,  0.1805]])


In [17]:
likelihood_fn_442 = gaussian_kde(likelihood_442[0,:,6:].T.cpu().numpy())
likelihood_fn_742 = gaussian_kde(likelihood_742[0,:,6:].T.cpu().numpy())
likelihood_fn_842 = gaussian_kde(likelihood_842[0,:,6:].T.cpu().numpy())

In [18]:
null_likelihood_fn_442 = gaussian_kde(null_hypothesis_442[0,:,6:].T.cpu().numpy())
null_likelihood_fn_742 = gaussian_kde(null_hypothesis_742[0,:,6:].T.cpu().numpy())
null_likelihood_fn_842 = gaussian_kde(null_hypothesis_842[0,:,6:].T.cpu().numpy())

In [19]:
log_likelihood_value_442 = likelihood_fn_442.logpdf(data[test_index]).item()
log_likelihood_value_742 = likelihood_fn_742.logpdf(data[test_index]).item()
log_likelihood_value_842 = likelihood_fn_842.logpdf(data[test_index]).item()

In [20]:
log_null_likelihood_value_442 = null_likelihood_fn_442.logpdf(data[test_index]).item()
log_null_likelihood_value_742 = null_likelihood_fn_742.logpdf(data[test_index]).item()
log_null_likelihood_value_842 = null_likelihood_fn_842.logpdf(data[test_index]).item()

In [21]:
print(f"Log Likelihood 442: {log_likelihood_value_442:.2f}")
print(f"Log Likelihood 742: {log_likelihood_value_742:.2f}")
print(f"Log Likelihood 842: {log_likelihood_value_842:.2f}")

Log Likelihood 442: 6.40
Log Likelihood 742: 0.44
Log Likelihood 842: 7.36


In [22]:
log_likelihoods = np.array([log_likelihood_value_442, log_likelihood_value_742, log_likelihood_value_842])
model_prob = scipy.special.softmax(log_likelihoods)

bayes_factor_null = [log_likelihood_value_442 - log_null_likelihood_value_442, log_likelihood_value_742 - log_null_likelihood_value_742, log_likelihood_value_842 - log_null_likelihood_value_842]
hypothesis_test = "could" if bayes_factor_null[model_prob.argmax()] > 0 else "could not"

hypothesis_test_strength = np.exp(bayes_factor_null[model_prob.argmax()])
if 1 < hypothesis_test_strength <= 3.2:
    hypothesis_test_strength = "barley"
elif 3.2 < hypothesis_test_strength <= 10:
    hypothesis_test_strength = "substantially"
elif 10 < hypothesis_test_strength <= 100:
    hypothesis_test_strength = "strongly"
elif 100 < hypothesis_test_strength:
    hypothesis_test_strength = "decisively"
else:
    hypothesis_test_strength = ""

print(f"Model {model_prob.argmax()+1} fits the data best " + 
      f"with a relative support of {100*model_prob.max():.1f}% among the considered models "+
      f"and {hypothesis_test} reject the null hypothesis {hypothesis_test_strength}.")

Model 3 fits the data best with a relative support of 72.2% among the considered models and could reject the null hypothesis strongly.
